In [13]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd

df =  pd.read_csv('teste_indicium_precificacao.csv')

# definindo um limite superior para preços baseado no percentil 99
limite_superior = np.percentile(df["price"], 99)
df_filtered = df[df["price"] <= limite_superior]


# transformando 'ultima_review' para datetime
df_filtered.loc[:, 'ultima_review'] = pd.to_datetime(df_filtered['ultima_review'])

# substituindo valores nulos de 'reviews_por_mes' por 0 (assumindo que sem reviews a média é 0)
df_filtered.loc[:, 'reviews_por_mes'] = df_filtered['reviews_por_mes'].fillna(0)

# selecionando as features relevantes
features = ['bairro_group', 'room_type', 'latitude', 'longitude', 'minimo_noites',
            'numero_de_reviews', 'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']

target = 'price'

# separando as variáveis categóricas e numéricas
cat_features = ['bairro_group', 'room_type']
num_features = ['latitude', 'longitude', 'minimo_noites', 'numero_de_reviews',
                'reviews_por_mes', 'calculado_host_listings_count', 'disponibilidade_365']

# pipeline de pré-processamento
preprocessor = ColumnTransformer(transformers=[
    ('num', 'passthrough', num_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), cat_features)
])

# criando modelo de regressão
model = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', RandomForestRegressor(n_estimators=500, random_state=42))
])

# separando os dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(df_filtered[features], df_filtered[target], test_size=0.2, random_state=42)

In [ ]:
# treinando o modelo
model.fit(X_train, y_train)

# fazendo previsões
y_pred = model.predict(X_test)

# avaliação do modelo
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

mae, rmse, r2

(44.64883398733306, np.float64(73.07930388312242), 0.5057559616341534)

In [ ]:
import joblib

# salvando o modelo treinado
joblib.dump(model, "modelo_predicao1.pkl")


['modelo_predicao1.pkl']

In [ ]:
# carregando o modelo salvo
modelo_carregado = joblib.load("modelo_predicao1.pkl")

# criando um exemplo de entrada com as características do apartamento
entrada = {
    "bairro_group": "Manhattan",
    "room_type": "Entire home/apt",
    "latitude": 40.75362,
    "longitude": -73.98377,
    "minimo_noites": 1,
    "numero_de_reviews": 45,
    "reviews_por_mes": 0.38,
    "calculado_host_listings_count": 2,
    "disponibilidade_365": 355
}

# convertendo a entrada em um DataFrame
entrada_df = pd.DataFrame([entrada])

# fazendo a previsão
preco_estimado = modelo_carregado.predict(entrada_df)[0]

print(f"Preço estimado: ${preco_estimado:.2f}")

Preço estimado: $245.17


In [16]:
# salvando os conjuntos de treino e teste separadamente
X_train.to_csv("X_train.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_train.to_csv("y_train.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

# concatenando X_train e y_train
train_data = pd.concat([X_train, y_train], axis=1)
train_data.to_csv("train.csv", index=False)

# concatenando X_test e y_test
test_data = pd.concat([X_test, y_test], axis=1)
test_data.to_csv("test.csv", index=False)

In [17]:
# carregando os dados de teste
test_data = pd.read_csv("test.csv")

# separando X_test e y_test
X_test = test_data.drop(columns=["price"])  # Substitua "price" pelo nome da coluna alvo
y_test = test_data["price"]

# fazendo previsões no X_test
y_pred = modelo_carregado.predict(X_test)

# comparando com os valores reais
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

# exibindo os resultados
print(f"MAE: {mae:.2f}")
print(f"RMSE: {rmse:.2f}")
print(f"R²: {r2:.2f}")

# criando um DataFrame com os resultados
comparacao = pd.DataFrame({"Real": y_test, "Previsto": y_pred})
print(comparacao.head())  # Mostrar as primeiras linhas

MAE: 44.65
RMSE: 5340.58
R²: 0.51
   Real  Previsto
0    50    54.632
1    50    60.526
2    59    58.390
3   150   236.276
4   175   163.008


In [18]:
first_test_samples = X_test[:5]  # Primeiros 5 exemplos de X_test
real_prices = y_test[:5]  # Primeiros 5 preços reais (y_test)

# Previsões para os primeiros exemplos
predicted_prices = y_pred[:5]


# Comparar as previsões com os valores reais
for i in range(5):
    print(f"\nExemplo {i+1}:")
    print(f"Preço real: {real_prices.iloc[i]:.2f}")
    print(f"Preço previsto: {predicted_prices[i]:.2f}")
    print(f"Diferença (Erro): {real_prices.iloc[i] - predicted_prices[i]:.2f}")


Exemplo 1:
Preço real: 50.00
Preço previsto: 54.63
Diferença (Erro): -4.63

Exemplo 2:
Preço real: 50.00
Preço previsto: 60.53
Diferença (Erro): -10.53

Exemplo 3:
Preço real: 59.00
Preço previsto: 58.39
Diferença (Erro): 0.61

Exemplo 4:
Preço real: 150.00
Preço previsto: 236.28
Diferença (Erro): -86.28

Exemplo 5:
Preço real: 175.00
Preço previsto: 163.01
Diferença (Erro): 11.99
